In [ ]:
## Tools

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_tavily import TavilySearch

api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv, description="Query arxiv papers")
print(arxiv.name)

api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
print(wiki.name)

tavily = TavilySearch()
# tavily.invoke("Provide me the recent Ai News")

In [47]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [48]:
## Combine all the tools in the list

tools = [arxiv, wiki, tavily]

## Initialize the LLM Model
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant")

llm_with_tools = llm.bind_tools(tools=tools)

## Workflow 

In [49]:
## State Schema
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage, HumanMessage  # Human Message or an AI Message
from typing import Annotated  # Labelling
from langgraph.graph.message import add_messages  # As a Reducers in Langgraph

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [ ]:
## Entire Chatbot with LangGraph

from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition 

# Node Definition
def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build graph
builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

# Add Edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm", 
    # - If the latest message from assistant is a tool call -> tool_condition routes to tools
    # - If the latest message from assistant is not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "tool_calling_llm")

# View Graph
graph = builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
result = graph.invoke({"messages": [HumanMessage(content="What is the recent AI news? and then tell me the recent research paper on quantum computing")]})
for r in result['messages']:
    r.pretty_print()